# Model Exploration Dash Prototype 

In [1]:
## Imports
#

import sys
sys.path.append('../GillesPy2')
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import callback_context
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
import os, ntpath
import numpy as np
from functools import reduce
import numpy as np
import gillespy2
from gillespy2 import Model, Species, Reaction, Parameter, RateRule, AssignmentRule, FunctionDefinition
from gillespy2 import EventAssignment, EventTrigger, Event
from sciope.stochmet.stochmet import StochMET


/home/smatthe2/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.manifold.t_sne module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.manifold. Anything that cannot be imported from sklearn.manifold is now part of the private API.



In [2]:
## GillesPy2 Model
#

class Dimerization(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="Dimerization")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="k_c", expression=0.005))
        self.add_parameter(Parameter(name="k_d", expression=0.08))

        # Species
        self.add_species(Species(name="Monomer", initial_value=30, mode="discrete"))
        self.add_species(Species(name="Dimer", initial_value=0, mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="r_creation", reactants={'Monomer': 2}, products={'Dimer': 1}, rate=self.listOfParameters["k_c"]))
        self.add_reaction(Reaction(name="r_dissociation", reactants={'Dimer': 1}, products={'Monomer': 2}, rate=self.listOfParameters["k_d"]))

        # Timespan
        self.timespan(np.linspace(0, 100, 101))
        
model = Dimerization()

In [3]:
from sciope.utilities.gillespy2 import wrapper
settings = {"number_of_trajectories":1}
simulator = wrapper.get_simulator(gillespy_model=model, run_settings=settings, species_of_interest=['Monomer', 'Dimer'])
expression_array = wrapper.get_parameter_expression_array(model)

In [4]:
from dask.distributed import Client
#from sciope.designs import latin_hypercube_sampling
#from sciope.utilities.summarystats.auto_tsfresh import SummariesTSFRESH
import sciope.utilities as util
import sciope.designs as designs
from sciope.utilities.priors.uniform_prior import UniformPrior

sampler = UniformPrior(expression_array*0.8, expression_array*1.5)

In [5]:
c = Client()

In [6]:
from sciope.utilities.summarystats.summary_base import SummaryBase

summaries = SummaryBase('summaries')
summaries.compute = lambda x: x

met = StochMET(simulator, sampler, summaries)

In [7]:
met.compute(n_points=50)

In [8]:
#First lets add some appropiate information about the model and features
met.data.configurations['parameter_labels'] = list(model.listOfParameters.keys())
met.data.configurations['species_labels'] = list(model.listOfSpecies.keys())
met.data.configurations['timepoints'] = model.tspan

In [9]:
data = met.data
data

In [10]:
import json

In [11]:
#### Start of Dash Setup
##

In [12]:
## Defaults
#

default_num_groups = 5
default_trajectory_selector = 'slider' # either 'dropdown' or 'slider'

In [13]:
## Helpers
#

def options_factory(iterable, label_accessor=lambda x: x, value_accessor=lambda x: x):
    """
    Factory function for creating a dash dropdown menu's 'options' attribute.

    Positional arguments:
    iterable -- any iterable data structure.

    Keyword arguments (optional):
    label_accessor -- function to retrieve the label from a single data point in iterable.
    value_accessor -- function to retrieve the value from a single data point in iterable.

    Returns a list of dictionaries with keys 'label' and 'value'.
    """
    return [
        { 'label': label_accessor(d), 'value': value_accessor(d) }
        for d in iterable
    ]

def get_zeroed_df(df):
    zero_df = df - df
    zero_df['time'] = df['time'].copy()
    return zero_df


def default_local_store(dfs, num_groups):
    """
    Initial client-side JSON storage for callbacks to refernece.
    """
    return {
        'dfs': [ df.to_json(orient='split') for df in dfs ],
        'group_assigns' : [None] * len(dfs),
        'group_data' : [get_zeroed_df(dfs[0]).to_json(orient='split')] * default_num_groups
    }

In [14]:
## App
#

external_stylesheets = [
    {
        'href': 'https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css',
        'rel': 'stylesheet',
        'integrity': 'sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO',
        'crossorigin': 'anonymous'
    }
]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)


In [17]:
# Create DataFrames

n_samp = len(data.x)
species = data.configurations['species_labels']
parameters = data.configurations['parameter_labels']
timepoints = data.configurations['timepoints']

inputs = {}
for i, p in enumerate(parameters):
    inputs[p] = data.x[:, i]
inputs['Sample'] = ['Sample_{}'.format(i) for i in range(n_samp)]
outputs = {'time':[]}
for i, tp in enumerate(timepoints):
    outputs['time'].append(tp)
for i in range(n_samp):
    for j, s in enumerate(species):
        lbl = '{}_{}'.format(s, i)
        outputs[lbl] = data.ts[i, 0, j, :]
input_df = pd.DataFrame(inputs)
output_df = pd.DataFrame(outputs)

In [18]:
# Create Div
fig1 = px.scatter(input_df, x=parameters[0], y=parameters[1], hover_name='Sample')
df_melt = output_df.melt(id_vars='time', value_vars=['{}_{}'.format(species[0], 0), '{}_{}'.format(species[1], 0)])
fig2 = px.line(df_melt, x='time', y ='value', color='variable')

app.layout = html.Div(children=[html.Div(
    dcc.Graph(id='parameter-space-graph', figure=fig1, clickData=None, hoverData=None, style={'width': '800'}), style={'display': 'inline-block'}),
    html.Div(
    dcc.Graph(id='time-series-graph', figure=fig2, style={'width': '800'}), style={'display': 'inline-block'})], 
                      style={'width': '100%', 'display': 'inline-block'})




In [19]:
# Create Callback for updateable output graph
@app.callback(
    Output(component_id='time-series-graph', component_property='figure'),
    [Input(component_id='parameter-space-graph', component_property='clickData'),
    Input(component_id='parameter-space-graph', component_property='selectedData')]
)
def update_time_series(clk_data, selectedData):
    df_melt = output_df.melt(id_vars='time', value_vars=['{}_{}'.format(species[0], 0), '{}_{}'.format(species[1], 0)])
    fig = px.line(df_melt, x='time', y ='value', color='variable')
    if clk_data:
        loc_1 = input_df.loc[input_df[parameters[0]] == clk_data['points'][0]['x']]
        loc_2 = input_df.loc[input_df[parameters[1]] == clk_data['points'][0]['y']]
        indx = input_df[parameters[0]][input_df[parameters[0]] == clk_data['points'][0]['x']].index.tolist()[0]
        df_melt = output_df.melt(id_vars='time', value_vars=['{}_{}'.format(species[0], indx), '{}_{}'.format(species[1], indx)])
        if loc_1.equals(loc_2):
            fig = px.line(df_melt, x='time', y='value', color='variable')
    if selectedData:
        df_melt = output_df.melt(id_vars='time', value_vars=['{}_{}'.format(species[0], 0), '{}_{}'.format(species[1], 0)])
        fig = px.line(df_melt, x='time', y ='value', color='variable')
        indices = []
        for point in selectedData['points']:
            loc_1 = input_df.loc[input_df[parameters[0]] == point['x']]
            loc_2 = input_df.loc[input_df[parameters[0]] == point['y']]
            indices.append(input_df[parameters[0]][input_df[parameters[0]] == point['x']].index.tolist()[0])
        values = []
        for i in indices:
            values.append('{}_{}'.format(species[0], i))
            values.append('{}_{}'.format(species[1], i))
        df_melt = output_df.melt(id_vars='time', value_vars=values)
        fig = px.line(df_melt, x='time', y='value', color='variable')
    return fig



In [20]:
## Run
#

# app.run_server(mode='inline', debug=True)
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [21]:
input_df

,k_c,k_d,Sample
0,0.007206,0.076020,Sample_0
1,0.007241,0.084180,Sample_1
2,0.005286,0.070085,Sample_2
3,0.006474,0.075054,Sample_3
4,0.006706,0.083763,Sample_4
5,0.004001,0.070963,Sample_5
6,0.006209,0.080238,Sample_6
7,0.005949,0.092760,Sample_7
8,0.005519,0.112370,Sample_8
9,0.006788,0.088778,Sample_9


In [22]:
output_df

,time,Monomer_0,Dimer_0,Monomer_1,Dimer_1,Monomer_2,Dimer_2,Monomer_3,Dimer_3,Monomer_4,...,Monomer_45,Dimer_45,Monomer_46,Dimer_46,Monomer_47,Dimer_47,Monomer_48,Dimer_48,Monomer_49,Dimer_49
0,0.0,30.0,0.0,30.0,0.0,30.0,0.0,30.0,0.0,30.0,...,30.0,0.0,30.0,0.0,30.0,0.0,30.0,0.0,30.0,0.0
1,1.0,26.0,2.0,26.0,2.0,28.0,1.0,22.0,4.0,28.0,...,22.0,4.0,26.0,2.0,26.0,2.0,18.0,6.0,20.0,5.0
2,2.0,18.0,6.0,18.0,6.0,26.0,2.0,16.0,7.0,20.0,...,22.0,4.0,24.0,3.0,22.0,4.0,20.0,5.0,18.0,6.0
3,3.0,18.0,6.0,18.0,6.0,16.0,7.0,16.0,7.0,18.0,...,14.0,8.0,16.0,7.0,22.0,4.0,18.0,6.0,20.0,5.0
4,4.0,14.0,8.0,14.0,8.0,18.0,6.0,14.0,8.0,16.0,...,12.0,9.0,14.0,8.0,14.0,8.0,14.0,8.0,18.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96.0,12.0,9.0,10.0,10.0,12.0,9.0,10.0,10.0,8.0,...,8.0,11.0,10.0,10.0,12.0,9.0,12.0,9.0,16.0,7.0
97,97.0,6.0,12.0,10.0,10.0,6.0,12.0,10.0,10.0,8.0,...,10.0,10.0,10.0,10.0,12.0,9.0,12.0,9.0,16.0,7.0
98,98.0,8.0,11.0,10.0,10.0,6.0,12.0,8.0,11.0,10.0,...,10.0,10.0,10.0,10.0,12.0,9.0,12.0,9.0,16.0,7.0
99,99.0,8.0,11.0,12.0,9.0,8.0,11.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,8.0,11.0,12.0,9.0,12.0,9.0
